In [1]:
import numpy as np

The initial Code:

In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def gradientDescent(X, Y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        theta: your final weight vector
        J: the final cost
    '''
    # get 'm', the number of rows in matrix X
    m = len(X)

    for _ in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(X, theta)
        
        # get the sigmoid of z
        h = sigmoid(z)

        # gradient
        gradient = (1/float(m)) * np.dot(X.T, (h - Y))

        # update the weights theta
        theta = theta - alpha *gradient

        # calculate J
        J = - (np.dot(Y.T, np.log(h)) + np.dot((1-Y).T, np.log(1-h))) / float(m)
        J = float(J)
        
    return theta,J 



# Errors:

- When coputing the loss function:

    J = - (np.dot(Y.T, np.log(h)) + np.dot((1-Y).T, np.log(1-h))) / m

    → If h becomes exactly 0 or 1, log(0) → -inf (not strictly divide-by-zero, but numerical error).

- The loop always runs num_iters times. If gradient becomes ~0 early, you’re wasting compute.
- Sigmoid can get overflow if z is extreme negative

    

# Solutions:

- avoid divide-by-zero error --> add epsilon

    eps = 1e-12
        J = - (np.dot(Y.T, np.log(h + eps)) + np.dot((1-Y).T, np.log(1-h + eps))) / len(X)

- stop when gradient norm is tiny:

    if np.linalg.norm(gradient) < tol:
        break

- More robust / numerically stable: avoid log(sigmoid) entirely

    z = X.dot(theta)
    z = np.clip(z, -500, 500)         # optional: protect np.exp from overflow
    J = np.mean(np.maximum(z, 0) - z * Y + np.log(1 + np.exp(-np.abs(z))))

-->This formula does not compute log(h) or log(1-h) directly and is stable for extreme z.

In [4]:
import numpy as np

def sigmoid(z):
    z = np.clip(z, -500, 500)   # tránh overflow
    return 1 / (1 + np.exp(-z))

def gradientDescent(X, Y, theta, alpha, num_iters, tol=1e-6, eps=1e-15):
    m = X.shape[0]

    for _ in range(num_iters):
        z = np.dot(X,theta)
        h = sigmoid(z)

        # gradient
        gradient = (X.T @ (h - Y)) / m

        if np.linalg.norm(gradient) < tol:  # early stopping
            break

        theta -= alpha * gradient

    # cost cuối cùng (công thức cũ có epsilon)
    J = - (np.dot(Y.T, np.log(h + eps)) + np.dot((1 - Y).T, np.log(1 - h + eps))) / m

    return theta, float(J)
